In [1]:
# ========================================
# library
# ========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_squared_error
%matplotlib inline
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import transformers
from transformers import LongformerTokenizer, LongformerModel,AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
import logging
from ast import literal_eval
import sys
from contextlib import contextmanager
import time
import random
from tqdm import tqdm
import os

2022-01-19 02:52:18.990788: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# ==================
# Constant
# ==================
ex = "019"
TRAIN_PATH = "../data/train.csv"
DATA_DIR = "../data/longformer-large-4096/"
if not os.path.exists(f"../output/exp/ex{ex}"):
    os.makedirs(f"../output/exp/ex{ex}")
    os.makedirs(f"../output/exp/ex{ex}/ex{ex}_model")
    
OUTPUT_DIR = f"../output/exp/ex{ex}"
MODEL_PATH_BASE = f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}"
LOGGER_PATH = f"../output/exp/ex{ex}/ex{ex}.txt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# ===============
# Configs
# ===============
SEED = 0
N_SPLITS = 5
SHUFFLE = True
num_workers = 4
BATCH_SIZE = 4

n_epochs = 6
max_len = 2048
weight_decay = 0.1
beta = (0.9, 0.98)
lr = 2e-5
num_warmup_steps_rate = 0.1

MODEL_PATH = 'allenai/longformer-large-4096'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [4]:
# ===============
# Functions
# ===============
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def setup_logger(out_file=None, stderr=True, stderr_level=logging.INFO, file_level=logging.DEBUG):
    LOGGER.handlers = []
    LOGGER.setLevel(min(stderr_level, file_level))

    if stderr:
        handler = logging.StreamHandler(sys.stderr)
        handler.setFormatter(FORMATTER)
        handler.setLevel(stderr_level)
        LOGGER.addHandler(handler)

    if out_file is not None:
        handler = logging.FileHandler(out_file)
        handler.setFormatter(FORMATTER)
        handler.setLevel(file_level)
        LOGGER.addHandler(handler)

    LOGGER.info("logger set up")
    return LOGGER


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
    
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2022-01-19 02:52:24,195 - INFO - logger set up


<RootLogger root (DEBUG)>

In [5]:
class TrainDataset(Dataset):
    def __init__(self, token,attentiona_mask,label=None):
        self.len = len(token)
        self.token = token
        self.attention_mask = attentiona_mask
        self.label = label
        #self.get_wids = get_wids # for validation

    def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        if self.label is not None:
            return {
              'token': torch.tensor(self.token[index], dtype=torch.long),
              'mask': torch.tensor(self.attention_mask[index], dtype=torch.long),
              "y":torch.tensor(self.label[index], dtype=torch.float32)
               }
        else:
            return {
              'token': torch.tensor(self.token[index], dtype=torch.long),
              'mask': torch.tensor(self.attention_mask[index], dtype=torch.long),
               }

    def __len__(self):
        return self.len

class custom_model(nn.Module):
    def __init__(self):
        super(custom_model, self).__init__()
        self.backbone = LongformerModel.from_pretrained(
            MODEL_PATH, 
        )
        
        #self.dropout = nn.Dropout(p=0.2)
        self.ln = nn.LayerNorm(1024)
        
        self.conv1= nn.Conv1d(1024, 512, kernel_size=3, padding=1)
        self.conv2= nn.Conv1d(1024, 512, kernel_size=9, padding=4)
        self.conv3= nn.Conv1d(1024, 512, kernel_size=15, padding=7)
        self.conv4= nn.Conv1d(1024, 512, kernel_size=31, padding=15)
        self.ln1 = nn.Sequential(nn.LayerNorm(512),
                                            nn.ReLU(),
                                            nn.Dropout(0.2))
        self.ln2 = nn.Sequential( nn.LayerNorm(512),
                                            nn.ReLU(),
                                            nn.Dropout(0.2))
        self.ln3 = nn.Sequential( nn.LayerNorm(512),
                                            nn.ReLU(),
                                            nn.Dropout(0.2))
        self.ln4 = nn.Sequential( nn.LayerNorm(512),
                                            nn.ReLU(),
                                            nn.Dropout(0.2))
        
        self.linear1 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear2 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear3 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear4 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear5 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear6 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear7 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,2),
        )
        self.linear8 = nn.Sequential(
            nn.Linear(2048,1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,1),
        )
    def forward(self, ids, mask):
        # pooler
        emb = self.backbone(ids, attention_mask=mask)["last_hidden_state"]
        output = self.ln(emb)
        output = output.permute((0, 2, 1)).contiguous()
        output1 = self.conv1(output)
        output1 = self.ln1(output1.permute((0, 2, 1)).contiguous())
        output2 = self.conv2(output)
        output2 = self.ln2(output2.permute((0, 2, 1)).contiguous())
        output3 = self.conv3(output)
        output3 = self.ln3(output3.permute((0, 2, 1)).contiguous())
        output4 = self.conv4(output)
        output4 = self.ln4(output4.permute((0, 2, 1)).contiguous())
        output_concat = torch.cat([output1,output2,output3,output4],axis=-1)
        output2_1 = self.linear1(output_concat)
        output2_2 = self.linear2(output_concat)
        output2_3 = self.linear3(output_concat)
        output2_4 = self.linear4(output_concat)
        output2_5 = self.linear5(output_concat)
        output2_6 = self.linear6(output_concat)
        output2_7= self.linear7(output_concat)
        output2_8 = self.linear8(output_concat)
        out = torch.cat(
            [output2_1,output2_2,output2_3,output2_4,
             output2_5,output2_6,output2_7,output2_8], axis=2)
        return out

In [6]:
target_map_rev = {0:'Lead', 1:'Position', 2:'Evidence', 3:'Claim', 4:'Concluding Statement',
             5:'Counterclaim', 6:'Rebuttal', 7:'blank'}

def get_preds_collate(dataset, verbose,text_ids, preds, preds_len):
    all_predictions = []

    for id_num in tqdm(range(len(preds))):
    
        # GET ID
        #if (id_num%100==0)&(verbose): 
        #    print(id_num,', ',end='')
        n = text_ids[id_num]
        max_len = int(preds_len[id_num])
        # GET TOKEN POSITIONS IN CHARS
        name = f'../data/{dataset}/{n}.txt'
        txt = open(name, 'r').read()
        tokens = tokenizer.encode_plus(txt, max_length=max_len, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
        off = tokens['offset_mapping']
    
        # GET WORD POSITIONS IN CHARS
        w = []
        blank = True
        for i in range(len(txt)):
            if (txt[i]!=' ')&(txt[i]!='\n')&(txt[i]!='\xa0')&(txt[i]!='\x85')&(blank==True):
                w.append(i)
                blank=False
            elif (txt[i]==' ')|(txt[i]=='\n')|(txt[i]=='\xa0')|(txt[i]=='\x85'):
                blank=True
        w.append(1e6)
            
        # MAPPING FROM TOKENS TO WORDS
        word_map = -1 * np.ones(max_len,dtype='int32')
        w_i = 0
        for i in range(len(off)):
            if off[i][1]==0: continue
            while off[i][0]>=w[w_i+1]: w_i += 1
            word_map[i] = int(w_i)
        
        # CONVERT TOKEN PREDICTIONS INTO WORD LABELS
        ### KEY: ###
        # 0: LEAD_B, 1: LEAD_I
        # 2: POSITION_B, 3: POSITION_I
        # 4: EVIDENCE_B, 5: EVIDENCE_I
        # 6: CLAIM_B, 7: CLAIM_I
        # 8: CONCLUSION_B, 9: CONCLUSION_I
        # 10: COUNTERCLAIM_B, 11: COUNTERCLAIM_I
        # 12: REBUTTAL_B, 13: REBUTTAL_I
        # 14: NOTHING i.e. O
        ### NOTE THESE VALUES ARE DIVIDED BY 2 IN NEXT CODE LINE
        pred = preds[id_num,]/2.0
    
        i = 0
        while i<max_len:
            prediction = []
            start = pred[i]
            if start in [0,1,2,3,4,5,6,7]:
                prediction.append(word_map[i])
                i += 1
                if i>=max_len: break
                while pred[i]==start+0.5:
                    if not word_map[i] in prediction:
                        prediction.append(word_map[i])
                    i += 1
                    if i>=max_len: break
            else:
                i += 1
            prediction = [x for x in prediction if x!=-1]
            if len(prediction)>4:
                all_predictions.append( (n, target_map_rev[int(start)], 
                                ' '.join([str(x) for x in prediction]) ) )
                
    # MAKE DATAFRAME
    df = pd.DataFrame(all_predictions)
    df.columns = ['id','class','predictionstring']
    
    return df


def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

def collate(d,train=True):
    mask_len = int(d["mask"].sum(axis=1).max())
    if train:
        return {"token" : d['token'][:,:mask_len],
                 "mask" : d['mask'][:,:mask_len],
                 "y" : d['y'][:,:mask_len],
                  "max_len" : mask_len}
    else:
        return {"token" : d['token'][:,:mask_len],
                 "mask" : d['mask'][:,:mask_len],
                  "max_len" : mask_len}

In [7]:
# ================================
# Main
# ================================
train = pd.read_csv(TRAIN_PATH)
IDS = train.id.unique()
id_array = np.array(IDS)

In [11]:
# ================================
# data load
# ================================
targets = np.load(DATA_DIR + f"targets_{max_len}.npy")
train_tokens = np.load(DATA_DIR + f"tokens_{max_len}.npy")
train_attention = np.load(DATA_DIR + f"attention_{max_len}.npy")

In [12]:
# ================================
# train
# ================================
with timer("longformer_large"):
    set_seed(SEED)
    oof = pd.DataFrame()
    oof_pred = np.ndarray((0,max_len,15))
    kf = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE, random_state=SEED)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(id_array)):
        print(f"fold{fold}:start")
        x_train_token, x_train_attention, y_train  = train_tokens[train_idx], train_attention[train_idx], targets[train_idx]
        x_val_token, x_val_attention, y_val  = train_tokens[valid_idx], train_attention[valid_idx], targets[valid_idx]
        train_val = train[train.id.isin(id_array[valid_idx])].reset_index(drop=True)
        
        # dataset
        train_ = TrainDataset( x_train_token, x_train_attention, y_train)
        val_ = TrainDataset( x_val_token, x_val_attention, y_val)
        
        # loader
        train_loader = DataLoader(dataset=train_, batch_size=BATCH_SIZE, shuffle = True ,pin_memory=True)
        val_loader = DataLoader(dataset=val_, batch_size=BATCH_SIZE, shuffle = False , pin_memory=True)
        
        # model
        model = custom_model()
        model = model.to(device)
        
        # optimizer, scheduler
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=lr,
                          betas=beta,
                          weight_decay=weight_decay,
                          )
        num_train_optimization_steps = int(len(train_loader) * n_epochs)
        num_warmup_steps = int(num_train_optimization_steps * num_warmup_steps_rate)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=num_warmup_steps,
                                                    num_training_steps=num_train_optimization_steps)
        
        criterion = nn.BCEWithLogitsLoss()
        best_val = 0
        
        for epoch in range(n_epochs):
            print(f"============start epoch:{epoch}==============")
            model.train() 
            val_losses_batch = []
            scaler = GradScaler()
            for i, d in tqdm(enumerate(train_loader),total=len(train_loader)):
                d = collate(d)
                ids = d['token'].to(device)
                mask = d['mask'].to(device)
                labels = d['y'].to(device)
                #labels = labels.unsqueeze(-1)
                optimizer.zero_grad()
                with autocast():
                    output = model(ids,mask)
                    loss = criterion(output[mask == 1], labels[mask == 1])
                scaler.scale(loss).backward()
                #torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
            
            y_pred2 = []
            val_preds = np.ndarray((0,max_len,15))
            val_len = np.ndarray(0)
            model.eval()  # switch model to the evaluation mode
            with torch.no_grad():  
                # Predicting on validation set
                
                for d in tqdm(val_loader,total=len(val_loader)):
                    # =========================
                    # data loader
                    # =========================
                    d = collate(d)
                    ids = d['token'].to(device)
                    mask = d['mask'].to(device)
                    with autocast():
                        outputs = model(ids, mask)
                    outputs = np.concatenate([outputs.sigmoid().detach().cpu().numpy(),
                                              np.zeros([len(outputs),max_len - d["max_len"],15])],axis=1)
                    val_preds = np.concatenate([val_preds, outputs], axis=0)
                    val_len = np.concatenate([val_len,np.array([d["max_len"] for i in range(len(ids))])],axis=0)
            val_preds_max = np.argmax(val_preds,axis=-1)
            oof_ = get_preds_collate( dataset='train', verbose=True, text_ids=id_array[valid_idx],
                                      preds = val_preds_max,preds_len=val_len)      
            # COMPUTE F1 SCORE
            f1s = []
            CLASSES = oof_['class'].unique()
            print()
            for c in CLASSES:
                pred_df = oof_.loc[oof_['class']==c].copy()
                gt_df = train_val.loc[train_val['discourse_type']==c].copy()
                f1 = score_feedback_comp(pred_df, gt_df)
                print(c,f1)
                f1s.append(f1)
            score = np.mean(f1s)
            LOGGER.info(f'{fold},{epoch}:{i},val_score:{score}')
            if best_val < score:
                print("save model weight")
                best_val = score
                best_val_preds = val_preds
                oof_best = oof_.copy()
                torch.save(model.state_dict(), MODEL_PATH_BASE + f"_{fold}.pth") # Saving current best model
        oof_best["fold"] = fold
        oof_best.to_csv(OUTPUT_DIR + f"/ex{ex}_oof_{fold}.csv",index=False)
        np.save(OUTPUT_DIR + f"/ex{ex}_oof_npy_{fold}.npy",best_val_preds)

fold0:start


Some weights of the model checkpoint at allenai/longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


============start epoch:0==============


100%|██████████| 3119/3119 [00:22<00:00, 139.68it/s]
2022-01-19 03:27:48,979 - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-01-19 03:27:48,980 - INFO - NumExpr defaulting to 8 threads.



Lead 0.8244919503826867
Claim 0.598008030453147
Evidence 0.6693015573383672
Position 0.6931382793451759
Concluding Statement 0.8076486179477471
Counterclaim 0.49957947855340623


...

100%|██████████| 3118/3118 [00:19<00:00, 159.19it/s]



Lead 0.8227176220806794
Position 0.6990810359231412
Evidence 0.7202504052316807
Claim 0.6076226179318963


2022-01-19 20:34:46,054 - INFO - 4,5:3118,val_score:0.6652452075375989


Concluding Statement 0.8241938422299143
Counterclaim 0.5394120153387303
Rebuttal 0.4434389140271493


2022-01-19 20:34:46,612 - INFO - [longformer_large] done in 63741 s


In [13]:
oof = pd.DataFrame()
for i in range(5):
    oof__ = pd.read_csv(OUTPUT_DIR + f"/ex{ex}_oof_{i}.csv")
    oof = pd.concat([oof,oof__]).reset_index(drop=True)
# COMPUTE F1 SCORE
f1s = []
CLASSES = oof['class'].unique()
for c in CLASSES:
    pred_df = oof.loc[oof['class']==c].copy()
    gt_df = train.loc[train['discourse_type']==c].copy()
    f1 = score_feedback_comp(pred_df, gt_df)
    print(c,f1)
    f1s.append(f1)
score = np.mean(f1s)
LOGGER.info(f'CV:{score}')

Lead 0.8218649517684887
Claim 0.6171306481638179
Evidence 0.7227554284936356
Position 0.7000402630519393
Concluding Statement 0.8329197377680655
Counterclaim 0.5368616691742633


2022-01-19 20:40:14,043 - INFO - CV:0.6696554829787125


Rebuttal 0.4560156824307768
